https://youtu.be/g-41MbfW4yw?t=513 
To get the attention scores, we scale the query-key dot product by the dim of the key (== dim of the query)

dim_of_key = key.size(-1)                                                       
attention_scores = torch.bmm(query, key.transpose(-2,-1))/sqrt(dim_of_key) 

where key (and query) are dim (batch_size, seq_len, embed_size//#heads)

At this point, in this attention head, we have pushed the full embedding through the attention head-specific Wq/Wk/Wv reduce
embed_size to embed_size//#heads 

In the multi-head attenttion layer, we give each head the full embedding. The head then "shrinks" it using its Wq/Wk/Wv and 
applies attention. The multi-head attention layer then reconstructs the head outputs to something of size embed_size

Back to scaled dot product:

We do this to "preserve variance".


Softmax does not like very large or very small numbers, will go to 1 or 0 respectively - test this
What we want is a nice distribution in what we feed into softmax
We want to preserve the variance (nice distribution) which is what scaling does

In [18]:
import torch
dim = 25
a = torch.randn(5*dim,dim) # randn fills with random numbers but variance is 1 (and mean is 0)
b = torch.randn(dim,5*dim)
ab = a@b
print(a.var())
print(b.var())
print(ab.var())
print(ab.shape)
ab = ab / ((dim)**.5)
print(ab.var())

tensor(0.9631)
tensor(1.0047)
tensor(24.2776)
torch.Size([125, 125])
tensor(0.9711)


In [ ]:
# Takeway is that our variance just blew up, which is what we don't want for stable learning

In [11]:
dim = 25
a = torch.randn(dim,5*dim) # randn fills with random numbers but variance is 1 (and mean is 0)
b = torch.randn(5*dim,dim)
ab = a@b
print(a.var())
print(b.var())
print(ab.var())
print(ab.shape)
ab = ab / ((5*dim)**.5)
print(ab.var())

tensor(0.9807)
tensor(1.0259)
tensor(126.3244)
torch.Size([25, 25])
tensor(1.0106)


In [ ]:
# So it's the "inner" dimension on which we multiply !!!!

In [19]:
print(a.norm())
print(b.norm())
ab.norm()

tensor(54.8608)
tensor(56.0311)


tensor(123.1824)

In [ ]:
# We can see that the norm is increase, but that does not matter, because softmax only cares about variance and not norm:

In [23]:
my_data = torch.tensor([1.,2.,10.,100.]) # This is a very high variance
torch.nn.functional.softmax(my_data, dim=-1)

tensor([1.0089e-43, 2.7465e-43, 8.1940e-40, 1.0000e+00])

In [ ]:
# This very high variance gives us a poor softmax result: the 100 is 1, all the rest is squashed to zero

In [24]:
my_data = torch.tensor([1.,.9,.7,1.2]) # This is a low variance
torch.nn.functional.softmax(my_data, dim=-1)

tensor([0.2586, 0.2340, 0.1916, 0.3158])

In [25]:
# Now we get usable softmax output
# New we increase the norm by adding 1000 to each element

my_data += 1000
torch.nn.functional.softmax(my_data, dim=-1)

tensor([0.2586, 0.2340, 0.1916, 0.3158])

In [ ]:
# We get the same softmax result!!!